Here, we get relevant data and parse a html converted from a pdf found at the url: of the top chinese characters:   

http://www.zein.se/patrick/3000char.html

In [18]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd
# Suppress just SettingWithCopyWarningda
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

### Part 1: get initial data of simpilifed and traditional characters, along with phrases/multi-character words in traditional

Loop through the tables of the html. With the exception of the first one, they follow a predictable pattern of definition, words from by multiple characters. 

We create an initial datasets: one with characters only (both simplifed and traditional), together another with multiple characters and their pronunciations. The character category(cat) is the index of the character (used for ranking relatively difficulty).

For alternative characters (mainly numbers) we only show it for the single character word

Special care is used to ensure merged traditional forms can be separated based on the word and meaning, such as (幹乾=>干)

In [19]:
#our key helper function to parse definitions and pronunciations apart given the raw main character form, etc
#returns a list of lists with the order ["cat", "word/character", "pronunciation", "definition", "code","alt"]. Code is if the word uses traditional/simplified (s,t,a). More contains extra information (in few cases where there is an alternative character)
# since many words may have compound simplified characters in this list, we will later need to convert again for the words that 
#note pronunciation is the the pronunciation of the character in the word
# are desired to only be written in traditional.
#Therefore, each row will be written twice, one for traditional one for simplified
#for single characters, we use the first pronunciation listed

def getbetween(input, start, end): #helper function to get the string in between two symbols (the first matching abtter)
    startind = input.find(start)
    #if the first one is not found, return empty:
    if startind == -1: 
        return ""
    #narrow down input
    input = input[startind+len(start):]
    endind = input.find(end)
    #if the first one is not found (but start is found), return the string starting from the end 
    if endind == -1:
        return input
    return input[:endind].strip() #otherwise, return 

#current helper function to get all text between characters (a list). Add space to the list, to not include whitespace
def getall_betweenchars(charlist, text):
    #if the length of the list is empty
    if len(charlist) == 0:
        return "" #empty string
    if len(charlist) == 1: #if there is only element, return the only one
        return charlist[0] #empty string
    #get the max and lowest indices 
    charlist.append(" ")
    max = 0 
    max_string = "" #string with the min index
    min = 1000000 #absurdly large
    min_string = "" 
    for char in charlist:
        #find the index of the charcter in the, text
        ind = text.find(char)
        if ind > max:
            max = ind
            max_string = char
        if ind < min:
            min = ind
            min_string = char
    l2 = len(max_string)
    return text[min:max+l2]

def get_wordList(main, des, cat):
    ret = [] #list to return

    simp = main.split("(")[0] #get the simplified form
    alt = getbetween(main,"(A",")") #set as empty string initially for the alternative form #check for alternative form, by default if there is non, this function should return an empty string 
    #the the traditional form, if any 
    trad = getbetween(main,"(F",")")
    if trad == "": #if function is empty, get to be the same as the simplified character
        trad = simp
    print("current simp. :"+simp+", current trad. :"+trad)
    frags = des.split("[") #split on [
    #iterate through diffenent fragments with pronunciations and descriptions within the definition
    #count the number of total definitions
    i = 0
    for frag in frags:
        #get the separate pronunciations
        pro = frag.split("]")[0]
        print("current pronunciation:"+pro)
        #if not empty
        if len(pro) > 0:
            #get rid of the part that is not part of the pronunciaiton
            defis = frag.replace(f"{pro}]", "")
            #print("pronunciation:", pro, ",definitions:", defis)
            defis = defis.split(";")#get separate defintions
            defis = [x for x in defis if x.strip()] #remove empty strings
#replace empty strings
            for d in range(0,len(defis)):
                defi =  defis[d].strip() #get a certain definition, with leading/ training whitespace remove. def is the simplified def. 
                print("current def:", defi)
                
                newTrad = False #store if at least 1 new trad. character is found
                #check for potentially multiple traditional characters
                def_parts = defi.split(", ")
                exclude = [] #append for parts to exclude in the traditional definitions (since we have to substract the separate traditional definition in this case)
                for part in def_parts:
                    print("current part:", part)
                    trad2 = getbetween(part,"(F",")") #modify the fragment string
                    if len(trad2) != 0: #if function is not empty, get to be the same as the simplified character. In this case we have a in line traditional character
                        newTrad = True
                        minidef = getbetween(part ,trad2, ", ")
                        minidef = minidef.split(")")[1].strip() #get the minidef (specialized definition between the new character and commas and if any)
                        nestchar = trad2.split(",")#for the few cases where there are multiple nested characters like (裏,裡), we set the 2nd one to be the variant
                        mainchar = nestchar[0]
                        #new_alternative character
                        new_alt = ""
                        print("minidef:", minidef, ", in def:", defi, "with character:", mainchar, ", and alternative:", new_alt)
                        if len(nestchar) > 1: new_alt = nestchar[1] #assign the variant to be the 2nd one
                        #in the case we have a subdef, we append an extra traditional entry 
                        new1 = [cat, mainchar, pro, minidef, "t",new_alt] #"cat", "word/charcter", "pronunciation", "definition", "code","alt"
                        ret.append(new1)
                    
                        ex_str = f", (F {trad2}) {minidef}"
                        exclude.append(ex_str) #string to replace in the original definition for the traditional definition
                traddef = defi  #create new traditional definition excluding the existing one
                print("exclusion list:", exclude)
                for ex in exclude:
                    traddef = traddef.replace(ex, "")

                
                chinese_pattern  = re.compile(r'[\u4E00-\u9FFF]+') #regex to get list of matching characters 
                match = chinese_pattern.findall(traddef)
                han = getall_betweenchars(match, traddef)   #helper to get all chars between the list of matching characters 
                print("found subcharacter:"+han+"|")
                if len(han)>1 and i!=0: #if the def definition contains multiple chinese characters, we do extra processing to extract the characters, and if this is not a new trad 
                    charind = defi.find(han) #find last index
                    char = han.strip() # the character for the word 
                    defi = defi[charind+len(han):] #modify the definition to be the remaining part
                    new1 = [cat, char, pro, defi, "s",alt] #"cat", "word/charcter", "pronunciation", "definition", "code","alt"
                    new2 = [cat, char, pro, traddef, "t",alt] #for now
                    ret.append(new1)
                    ret.append(new2)
                    #in this case, we append as so 
                    print("ran if"+str(new2))
                elif newTrad==False: #assume that there is only one character to the definition, and no traditional characters found
                    new1 = [cat, simp, pro, defi, "s",alt] #"cat", "word/charcter", "pronunciation", "definition", "code","alt"
                    new2 = [cat, trad, pro, defi, "t",alt]
                    ret.append(new1)
                    ret.append(new2)
                    print("ran elif 1"+str(new2))
                else: 
                    print("ran else (diff traditional and simplified definition)")

                    new1 = [cat, simp, pro, defi, "s",alt] #simplified definotion is as usual"
                    new2 = [cat, trad, pro, traddef, "t",alt]
                    ret.append(new1)
                    ret.append(new2)
                #print("character:"+char+", definition:"+defi)
                i+= 1

    return ret 

In [20]:
link = "./main_char_set-converted.html"

with open(link, 'r') as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')
tables = soup.find_all("table")  
grand_list = [] #list used to create the dataframe in the end
#count the number of tables 
index = 0;
for table in tables:
    rows = table.find_all("tr") #extract all tables
    for row in rows: 
        # Extract all cells (td or th)
        cols = row.find_all(["td"]) #print the number of cols (exclude the header) 
        #valid character columns have 3 columns 
        if len(cols) == 3:
            #get the number column 
            col1 = cols[0].get_text()
            #get the char column 
            col2 = cols[1].get_text()
            #get description column, this is complicated and is where we split the characters from the words, using a helper function
            col3 = cols[2].get_text()
            #call our key helper function that retruns a list to be appended to the grand_list 
            res = get_wordList(col2, col3, cat=index)
            grand_list.append(res)
            index += 1
print("total number of characters listed:", index)

current simp. :Caractère, current trad. :Caractère
current pronunciation:Prononciation et explications
current def: Prononciation et explications
current part: Prononciation et explications
exclusion list: []
found subcharacter:|
ran elif 1[0, 'Caractère', 'Prononciation et explications', 'Prononciation et explications', 't', '']
current simp. :的, current trad. :的
current pronunciation:
current pronunciation:de
current def: <grammatical particle marking genitive as well as simple and composed adjectives>
current part: <grammatical particle marking genitive as well as simple and composed adjectives>
exclusion list: []
found subcharacter:|
ran elif 1[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 't', '']
current def: 我的 wǒde my
current part: 我的 wǒde my
exclusion list: []
found subcharacter:我的|
ran if[1, '我的', 'de', '我的 wǒde my', 't', '']
current def: 高的 gāode high, tall
current part: 高的 gāode high
current part: tall
exclusion list: []


In [21]:
print(len(grand_list))
grand_list

2431


[[[0,
   'Caractère',
   'Prononciation et explications',
   'Prononciation et explications',
   's',
   ''],
  [0,
   'Caractère',
   'Prononciation et explications',
   'Prononciation et explications',
   't',
   '']],
 [[1,
   '的',
   'de',
   '<grammatical particle marking genitive as well as simple and composed adjectives>',
   's',
   ''],
  [1,
   '的',
   'de',
   '<grammatical particle marking genitive as well as simple and composed adjectives>',
   't',
   ''],
  [1, '我的', 'de', ' wǒde my', 's', ''],
  [1, '我的', 'de', '我的 wǒde my', 't', ''],
  [1, '高的', 'de', ' gāode high, tall', 's', ''],
  [1, '高的', 'de', '高的 gāode high, tall', 't', ''],
  [1, '是的', 'de', " shìde that's it, that's right", 's', ''],
  [1, '是的', 'de', "是的 shìde that's it, that's right", 't', ''],
  [1, '是...的', 'de', 'shì...de one who...', 's', ''],
  [1, '是...的', 'de', '是...的 shì...de one who...', 't', ''],
  [1,
   '他是说汉语的',
   'de',
   '. Tā shì shuō Hànyǔde. He is one who speaks Chinese.',
   's',
   ''],


As expected, the list has 2431 subcomponents, but to effectively create the dataframe we need to remove a layer. 

In [22]:
master_list = []
for l1 in grand_list:
    for l2 in l1:
        print(l2)
        master_list.append(l2)

[0, 'Caractère', 'Prononciation et explications', 'Prononciation et explications', 's', '']
[0, 'Caractère', 'Prononciation et explications', 'Prononciation et explications', 't', '']
[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 's', '']
[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 't', '']
[1, '我的', 'de', ' wǒde my', 's', '']
[1, '我的', 'de', '我的 wǒde my', 't', '']
[1, '高的', 'de', ' gāode high, tall', 's', '']
[1, '高的', 'de', '高的 gāode high, tall', 't', '']
[1, '是的', 'de', " shìde that's it, that's right", 's', '']
[1, '是的', 'de', "是的 shìde that's it, that's right", 't', '']
[1, '是...的', 'de', 'shì...de one who...', 's', '']
[1, '是...的', 'de', '是...的 shì...de one who...', 't', '']
[1, '他是说汉语的', 'de', '. Tā shì shuō Hànyǔde. He is one who speaks Chinese.', 's', '']
[1, '他是说汉语的', 'de', '他是说汉语的. Tā shì shuō Hànyǔde. He is one who speaks Chinese.', 't', '']
[1, '目的', 'dì', ' mùdì g

Saving to a new csv file. First, remove the last two and first two entries from the, its clean its garbage

In [23]:
master_list = master_list[2:len(master_list)-2]
for l in master_list:
    print(l)

[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 's', '']
[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 't', '']
[1, '我的', 'de', ' wǒde my', 's', '']
[1, '我的', 'de', '我的 wǒde my', 't', '']
[1, '高的', 'de', ' gāode high, tall', 's', '']
[1, '高的', 'de', '高的 gāode high, tall', 't', '']
[1, '是的', 'de', " shìde that's it, that's right", 's', '']
[1, '是的', 'de', "是的 shìde that's it, that's right", 't', '']
[1, '是...的', 'de', 'shì...de one who...', 's', '']
[1, '是...的', 'de', '是...的 shì...de one who...', 't', '']
[1, '他是说汉语的', 'de', '. Tā shì shuō Hànyǔde. He is one who speaks Chinese.', 's', '']
[1, '他是说汉语的', 'de', '他是说汉语的. Tā shì shuō Hànyǔde. He is one who speaks Chinese.', 't', '']
[1, '目的', 'dì', ' mùdì goal', 's', '']
[1, '目的', 'dì', '目的 mùdì goal', 't', '']
[1, '的', 'dí', 'true, real', 's', '']
[1, '的', 'dí', 'true, real', 't', '']
[1, '的确', 'dí', ' díquè certainly', 's', '']
[1, '的确

We now save to a dataframe 

In [24]:
data =  pd.DataFrame(master_list, columns=["cat", "word/character", "pronunciation", "definition", "code","alt"])
data.to_csv("main_list.csv", index=False)


### Part 2: 

Creating a map of simplified to traditional characters so we can effectively edit multicharacter words here. Note that this is map from simplified characters to a list of traditional ones.

In [25]:
#isolate all rows of single characters 
single_characters = data[data["word/character"].str.len() == 1]
print(len(single_characters))
single_characters.head(n=1)
#save 
single_characters.to_csv("character_list.csv", index=False)

5728


In [26]:
#iteration based on category
cats = single_characters["cat"].unique()
StoTmap = {} #character map

for cat in cats:
    subset =  single_characters[single_characters["cat"] == cat]
    subset = subset.drop_duplicates(subset=['word/character']) #get the unique character rows 
    char_list = set(subset["word/character"]) #get the of characters
    #the simplified character is the one where the code is "s"
    simp = subset[subset["code"]=="s"].iloc[0]["word/character"]
    trad = list(subset[subset["code"]=="t"]["word/character"]) #get list of purely traditional ch
    if len(trad) == 0: #if the list of traditional characters is empty, then they must be the same
        StoTmap[simp] = [simp]
    else:
        StoTmap[simp] = list(trad)

In [27]:
for key in StoTmap:
    print(key,",", StoTmap[key])

的 , ['的']
一 , ['一']
是 , ['是']
不 , ['不']
了 , ['了']
人 , ['人']
我 , ['我']
在 , ['在']
有 , ['有']
他 , ['他']
这 , ['這']
中 , ['中']
大 , ['大']
来 , ['來']
上 , ['上']
国 , ['國']
个 , ['个']
到 , ['到']
说 , ['說']
们 , ['們']
为 , ['為']
子 , ['子']
和 , ['龢']
你 , ['你']
地 , ['地']
出 , ['齣']
道 , ['道']
也 , ['也']
时 , ['時']
年 , ['年']
得 , ['得']
就 , ['就']
那 , ['那']
要 , ['要']
下 , ['下']
以 , ['以']
生 , ['生']
会 , ['會']
自 , ['自']
着 , ['著', '招']
去 , ['去']
之 , ['之']
过 , ['過']
家 , ['家']
学 , ['學']
对 , ['對']
可 , ['可']
她 , ['她']
里 , ['裏']
后 , ['後']
小 , ['小']
么 , ['麼']
心 , ['心']
多 , ['多']
天 , ['天']
而 , ['而']
能 , ['能']
好 , ['好']
都 , ['都']
然 , ['然']
没 , ['沒']
日 , ['日']
于 , ['於']
起 , ['起']
还 , ['還']
发 , ['發', '髮']
成 , ['成']
事 , ['事']
只 , ['祇', '隻']
作 , ['作']
当 , ['當', '噹']
想 , ['想']
看 , ['看']
文 , ['文']
无 , ['無']
开 , ['開']
手 , ['手']
十 , ['十']
用 , ['用']
主 , ['主']
行 , ['行']
方 , ['方']
又 , ['又']
如 , ['如']
前 , ['前']
所 , ['所']
本 , ['本']
见 , ['見']
经 , ['經']
头 , ['頭']
面 , ['麵']
公 , ['公']
同 , ['衕']
三 , ['三']
已 , ['已']
老 , ['老']
从 , ['從']
动 , ['動']


get the inverse map. this is easy since each traditional character maps to just one simplified one. 

In [28]:
TtoS = {}
for key in StoTmap:
    Tlist = StoTmap[key]
    for Tchar in Tlist:
        TtoS[Tchar] = key 

Now, attempt to convert most compound words and definition characters to traditional automatically using this dictionary (for cases where the category is traditional). For cases of multiple mappings, we skip and print out a message highlighting the index to correct.

In [29]:
charmap = StoTmap
#store failed indices for the word/character column
failed_chars = []
#store failed indices for the definition column
definition_chars = []
def convert(data): #    
    for index, row in data.iterrows():
        #first convert characters 
        code = row["code"]
        word = row["word/character"]
        defi = row["definition"]
        #check if variables are strings or not
        if (isinstance(defi, str) == False):
            print(f"definition at index {index} is not a string: {defi}, for char {word}")
            print("converted to:", str(defi))

        if code == "t":
            #convert the word first 
            new_word = "" #new string 
            for char in str(word):
                #check if in dict 
                if char in charmap:
                    char_list = charmap[char]
                    if (len(char_list)>1): 
                        print("failed to word/character column at index:", index,", with char:", char, "failing", ", context is:", word)
                        failed_chars.append(index)
                        new_word += char #assign back to old word in this case
                    else: 
                        new_word += char_list[0] #increment 
                else:
                    new_word += char #increment like usual 
            data.at[index,"word/character"] = new_word #assign 

            #then, convert the description characters. use the same character if possible
            new_defi = "" #new string 
            for char in str(defi):
                #check if in dict 
                if char in charmap:
                    char_list = charmap[char]
                    if (len(char_list)>1): 
                        print("failed to definition column at index:", index,", with char:", char, "failing", ", context is:", word)
                        definition_chars.append(index)
                        new_defi += char #increment like usual 
                    else: 
                        new_defi += char_list[0] #increment 
                else:
                    new_defi += char #increment like usual 
            data.at[index,"definition"] = new_defi #assign 

    return data
data = convert(data)
data.to_csv("./main_list.csv", index=False)

failed to word/character column at index: 53 , with char: 只 failing , context is: 你来了! Nǐ láile! You have come! 我累了! Wǒ lèile! I've gottentired! 那好了! Nà hǎole! That's OK (now)! 我只请了一位客人
failed to definition column at index: 53 , with char: 只 failing , context is: 你来了! Nǐ láile! You have come! 我累了! Wǒ lèile! I've gottentired! 那好了! Nà hǎole! That's OK (now)! 我只请了一位客人
failed to word/character column at index: 391 , with char: 干 failing , context is: 天气 tiānqì weather{天干 tiāngān the Ten Heavenly Stems: 甲乙丙丁戊己庚辛壬癸 jiǎ-yǐ-bǐng-dīng-wù-jǐ-gēng-xīn-rén-guǐ (these characters aresometimes used as ordinal numbers}{Compare with 夫 fū man and 无
failed to definition column at index: 391 , with char: 干 failing , context is: 天气 tiānqì weather{天干 tiāngān the Ten Heavenly Stems: 甲乙丙丁戊己庚辛壬癸 jiǎ-yǐ-bǐng-dīng-wù-jǐ-gēng-xīn-rén-guǐ (these characters aresometimes used as ordinal numbers}{Compare with 夫 fū man and 无
failed to word/character column at index: 419 , with char: 当 failing , context is: 当然
failed t

We first manually remove 2 clearly badly parsed results from the dataset. With the "word/character" 幹) do{天干 tiāngān the Ten Heavenly Stems: 甲乙丙丁戊己庚辛壬癸 jiǎ-yǐ-bǐng-dīng-wù-jǐ-gēng-xīn-rén-guǐ (these characters aresometimes used as ordinal numbers}{Compare with 千 qiān thousand and 于.

Run again 

In [30]:
data = pd.read_csv("./main_list.csv")

Print out failed indices text for word/character column. From the outputs it is clear that the all failures are a single character, so we can replace with one.
Once we fix this column, the rest is easier, which is obvious by replacing corresponding character in the definition column


In [31]:
k1 = "word/character" #first key

i = 53
data.at[i,k1] = data.at[i,k1].replace("只","祇")

i = 463
data.at[i,k1] = data.at[i,k1].replace("只","隻")
i = 605
data.at[i,k1] = data.at[i,k1].replace("发","髮")
i = 687
data.at[i,k1] = data.at[i,k1].replace("发","發")

i = 1573
data.at[i,k1] = data.at[i,k1].replace("着","著")
i = 1835
data.at[i,k1] = data.at[i,k1].replace("发","發")

# 2349 has no change
i = 2479
data.at[i,k1] = data.at[i,k1].replace("尽","儘")
i = 2701
data.at[i,k1] = data.at[i,k1].replace("干","幹")

i = 3013
data.at[i,k1] = data.at[i,k1].replace("着","著")

i = 3385
data.at[i,k1] = data.at[i,k1].replace("干","幹")

i = 4177
data.at[i,k1] = data.at[i,k1].replace("采","採")
i = 4327
data.at[i,k1] = data.at[i,k1].replace("发","發")
i = 4493
data.at[i,k1] = data.at[i,k1].replace("发","發")
i = 4843
data.at[i,k1] = data.at[i,k1].replace("汇","彙")
i = 4859
data.at[i,k1] = data.at[i,k1].replace("发","發")
i = 4995
data.at[i,k1] = data.at[i,k1].replace("钟","鐘")
i = 5235
data.at[i,k1] = data.at[i,k1].replace("干","乾")
i = 5639
data.at[i,k1] = data.at[i,k1].replace("汇","匯")
i = 5641
data.at[i,k1] = data.at[i,k1].replace("汇","彙")

i = 5891
data.at[i,k1] = data.at[i,k1].replace("干","乾")
i = 6005
data.at[i,k1] = data.at[i,k1].replace("冲","衝")
i = 6071
data.at[i,k1] = data.at[i,k1].replace("发","發")
# 6635 is unchanged
i = 7201
data.at[i,k1] = data.at[i,k1].replace("干","乾")

i = 8007
data.at[i,k1] = data.at[i,k1].replace("钟","鐘")

i = 8203
data.at[i,k1] = data.at[i,k1].replace("干","乾")
i = 8631
data.at[i,k1] = data.at[i,k1].replace("干","著")

i = 8631
data.at[i,k1] = data.at[i,k1].replace("干","乾")

i = 9239
data.at[i,k1] = data.at[i,k1].replace("干","乾")
i = 9573
data.at[i,k1] = data.at[i,k1].replace("冲","衝")
i = 10407
data.at[i,k1] = data.at[i,k1].replace("干","乾")

Similarly, repair the destination column 

In [32]:
k1 = "definition" #first key

#429, 1567 no change 
i = 447
data.at[i,k1] = data.at[i,k1].replace("发","髮")

i = 1569 #special replacement for 幹
data.at[i,k1] = data.at[i,k1].replace("(F 幹) ","do")

i = 2177 
data.at[i,k1] = data.at[i,k1].replace("历","歷")
i = 2179 
data.at[i,k1] = data.at[i,k1].replace("历","曆")
i = 2183 
data.at[i,k1] = data.at[i,k1].replace("历","歷")
#2189, 3003-3009 no change 
i = 3333 
data.at[i,k1] = data.at[i,k1].replace("只","祇")

i = 3557 
data.at[i,k1] = data.at[i,k1].replace("采","埰")

i = 4173 
data.at[i,k1] = data.at[i,k1].replace("钟","鐘")

i = 5647 
data.at[i,k1] = data.at[i,k1].replace("历","歷")

i = 7443 
data.at[i,k1] = data.at[i,k1].replace("发","發")



finally, drop rows with invalid characters, and replace (F: with a more clear traditional mark) and (S with a more clear simplified mark)

Scratch: testing regex

In [33]:
text = "(F 幹) do{天干 tiāngān the Ten Heavenly Stems: 甲乙丙丁戊己庚辛壬癸 jiǎ-yǐ-bǐng-dīng-wù-jǐ-gēng-xīn-rén-guǐ (these characters aresometimes used as ordinal numbers}{Compare with 千 qiān thousand and 于 yú at}"
trad2 = getbetween(text,"(F",")") #modify the fragment string
print(trad2)

幹
